In [1]:
from basic_import import *
from load_data import load_data_thres, load_all_data
from binaryclass_memory import *
from sklearn.metrics import balanced_accuracy_score

In [8]:
%run thresholds.py

In [ ]:
%run ValidationNN.py

# Visualization data

In [ ]:
for i,charac in enumerate(charac_matrix):
    print(names[i])
    visualize_charac(charac)

In [ ]:
visualize_charac(charac_matrix[3])